# Text summarization: TextRank

With enormous amount of data surrounding us, it is important to be able to extract the most important information from it. In this notebook, we focus on one such information extraction algorithm from text. The questions which we address are: 
   * Given a document, which are the most important lines in it?   
   * Given a document, which are the most important key-words in it? 
In order to be able to answer these questions, we need to understand a lot of thing. 